In [22]:
import pandas as pd
import os
from ast import literal_eval

from sklearn.linear_model import LassoCV

import statsmodels.formula.api as smf

In [23]:
excel_file = '../working-csvs/20240114 Participants Key.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)

data_sheet_names = list(all_sheets.keys())[1:]

dfs = []

for sheet_name in data_sheet_names:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=1)
    for col in ['Name', 'Number', 'Affiliation']:
        alt_name = f'Participant\n{col}'
        if alt_name in sheet.columns:
            sheet[f'Participant {col}'] = sheet[alt_name]
            sheet.drop(columns=alt_name, inplace=True)
    sheet['MTGDATE'] = sheet_name
    dfs.append(sheet)

pkey = pd.concat(dfs)

In [24]:
pkey

,Participant Number,Participant Name,Participant Affiliation,Vote,Note,Chair,MTGDATE
0,1,Frederic Mishkin,Board,1.0,NaN,0.0,20071031
1,2,Gary Stern,Minneapolis,0.0,NaN,0.0,20071031
2,3,Timothy Geithner,New York,1.0,NaN,0.0,20071031
3,4,William Poole,St. Louis,1.0,NaN,0.0,20071031
4,5,Randall Kroszner,Board,1.0,NaN,0.0,20071031
...,...,...,...,...,...,...,...
12,13,Eric Rosengren,Boston,0.0,NaN,0.0,20181219
13,14,Loretta Mester,Cleveland,1.0,NaN,0.0,20181219
14,15,Patrick Harker,Philadelphia,0.0,NaN,0.0,20181219
15,16,Charles Evans,Chicago,0.0,NaN,0.0,20181219


In [25]:
proj = pd.read_excel('../working-csvs/20240114 FomcProjections.xlsx')

In [26]:
proj.dtypes

MTGDATE      int64
ID           int64
TARGET      object
HORIZON      int64
GDP        float64
UN         float64
PCE        float64
COREPCE    float64
FFD        float64
dtype: object

In [27]:
pkey['MTGDATE'] = pkey['MTGDATE'].astype(int)

In [28]:
pkey.dtypes

Participant Number           int64
Participant Name            object
Participant Affiliation     object
Vote                       float64
Note                        object
Chair                      float64
MTGDATE                      int64
dtype: object

In [29]:
proj = proj.merge(pkey, left_on=['MTGDATE', 'ID'], right_on=['MTGDATE', 'Participant Number'])

In [30]:
proj

,MTGDATE,ID,TARGET,HORIZON,GDP,UN,PCE,COREPCE,FFD,Participant Number,Participant Name,Participant Affiliation,Vote,Note,Chair
0,20071031,1,2007,1,2.4,4.7,3.0,1.8,NaN,1,Frederic Mishkin,Board,1.0,NaN,0.0
1,20071031,1,2008,5,1.7,4.8,1.8,1.9,NaN,1,Frederic Mishkin,Board,1.0,NaN,0.0
2,20071031,1,2009,9,2.2,4.8,1.9,2.0,NaN,1,Frederic Mishkin,Board,1.0,NaN,0.0
3,20071031,1,2010,13,2.2,4.8,2.0,2.0,NaN,1,Frederic Mishkin,Board,1.0,NaN,0.0
4,20071031,2,2007,1,2.5,4.7,3.0,1.9,NaN,2,Gary Stern,Minneapolis,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,20181219,17,2018,1,3.1,3.7,1.8,1.9,2.38,17,Raphael Bostic,Atlanta,1.0,NaN,0.0
2748,20181219,17,2019,5,2.3,3.5,2.0,2.1,2.63,17,Raphael Bostic,Atlanta,1.0,NaN,0.0
2749,20181219,17,2020,9,1.8,3.7,2.0,2.0,2.88,17,Raphael Bostic,Atlanta,1.0,NaN,0.0
2750,20181219,17,2021,13,1.8,3.9,2.0,2.0,2.88,17,Raphael Bostic,Atlanta,1.0,NaN,0.0


In [31]:
proj['year'] = ((proj['MTGDATE'] - proj['MTGDATE'] % 10000)/10000).astype(int)
proj['md'] = (proj['MTGDATE'] - proj['year'] * 10000)
proj['m'] = ((proj['md'] - proj['md'] % 100)/100).astype(int)

In [32]:
proj['speaker'] = proj['Participant Name'].map(lambda x : x.split()[-1].lower())

In [33]:
mfdff = pd.read_csv('../working-csvs/mfdff.csv', index_col=0)

In [34]:
mfdff['date'] = pd.to_datetime(mfdff['date_x'])

In [35]:
mfdff['year'] = mfdff['date'].dt.year
mfdff['m'] = mfdff['date'].dt.month

In [36]:
proj[['year', 'm']]

,year,m
0,2007,10
1,2007,10
2,2007,10
3,2007,10
4,2007,10
...,...,...
2747,2018,12
2748,2018,12
2749,2018,12
2750,2018,12


In [37]:
mfdff['norm_svect'] = mfdff['norm_svect'].map(lambda x : [float(item[:-1]) for item in x[1:-1].split()])

In [38]:
mfdff['diff_exp_norm'] = mfdff['diff_exp_norm'].map(lambda x : [float(item[:-1]) for item in x[1:-1].split()])

In [39]:
mfdff[[f'tprob_{i}' for i in range(45)]] = mfdff['norm_svect'].to_list()

In [40]:
df = proj.merge(mfdff, left_on=['year', 'm', 'speaker'], right_on=['year', 'm', 'lname'], how='left', indicator=True)

In [41]:
df['_merge'].value_counts()

_merge
both          5151
left_only       59
right_only       0
Name: count, dtype: int64

Check these

In [42]:
df[(df['_merge'] == 'left_only') & (df['year'] < 2018)].speaker

1317      fisher
1318      fisher
1319      fisher
1320      fisher
1513       moore
1514       moore
1515       moore
1516       moore
1517       moore
1720       moore
1721       moore
1722       moore
1723       moore
3101    pianalto
3102    pianalto
3103    pianalto
3104    pianalto
4232     gooding
4233     gooding
4234     gooding
4235     gooding
Name: speaker, dtype: object

In [43]:
df['HORIZON'].value_counts()

HORIZON
99    1031
1      358
5      358
9      358
13     358
3      297
7      297
11     297
2      292
6      292
10     292
4      284
8      284
12     284
14     128
Name: count, dtype: int64

In [44]:
def horiz_mapper(horiz):
    if horiz <= 3:
        return '1Q'
    if horiz > 3 and horiz <= 6:
        return '2Q'
    if horiz > 6 and horiz <= 9:
        return '3Q'
    if horiz > 9 and horiz <= 12:
        return '4Q'
    if horiz > 12 and horiz <= 14:
        return '5Q'
    if horiz >= 99:
        return 'LR'


In [45]:
df['HORIZON_condensed'] = df['HORIZON'].map(horiz_mapper)

In [46]:
df

,MTGDATE,ID,TARGET,HORIZON,GDP,UN,PCE,COREPCE,FFD,Participant Number,...,tprob_37,tprob_38,tprob_39,tprob_40,tprob_41,tprob_42,tprob_43,tprob_44,_merge,HORIZON_condensed
0,20071031,1,2007,1,2.4,4.7,3.0,1.8,NaN,1,...,-0.883743,-1.403502,-1.302201,0.2,-0.166723,0.326487,0.251383,-0.217420,both,1Q
1,20071031,1,2007,1,2.4,4.7,3.0,1.8,NaN,1,...,0.200000,0.175300,-1.228999,0.0,0.341441,0.000000,0.024013,0.260701,both,1Q
2,20071031,1,2008,5,1.7,4.8,1.8,1.9,NaN,1,...,-0.883743,-1.403502,-1.302201,0.2,-0.166723,0.326487,0.251383,-0.217420,both,2Q
3,20071031,1,2008,5,1.7,4.8,1.8,1.9,NaN,1,...,0.200000,0.175300,-1.228999,0.0,0.341441,0.000000,0.024013,0.260701,both,2Q
4,20071031,1,2009,9,2.2,4.8,1.9,2.0,NaN,1,...,-0.883743,-1.403502,-1.302201,0.2,-0.166723,0.326487,0.251383,-0.217420,both,3Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,20181219,17,2020,9,1.8,3.7,2.0,2.0,2.88,17,...,0.000000,1.028212,0.056627,0.0,-0.288675,0.000000,0.300605,2.711210,both,3Q
5206,20181219,17,2021,13,1.8,3.9,2.0,2.0,2.88,17,...,0.000000,0.661252,0.044623,0.0,0.000000,0.000000,-0.744847,0.584086,both,5Q
5207,20181219,17,2021,13,1.8,3.9,2.0,2.0,2.88,17,...,0.000000,1.028212,0.056627,0.0,-0.288675,0.000000,0.300605,2.711210,both,5Q
5208,20181219,17,LR,99,1.8,4.2,2.0,NaN,3.00,17,...,0.000000,0.661252,0.044623,0.0,0.000000,0.000000,-0.744847,0.584086,both,LR


In [47]:
fore1Q = df[df['HORIZON_condensed'] == '1Q']

In [48]:
tprob_cols = [f'tprob_{i}' for i in range(45)]

In [49]:
tprob_part = ' + '.join(tprob_cols)

In [50]:
'UN ~ ' + tprob_part

'UN ~ tprob_0 + tprob_1 + tprob_2 + tprob_3 + tprob_4 + tprob_5 + tprob_6 + tprob_7 + tprob_8 + tprob_9 + tprob_10 + tprob_11 + tprob_12 + tprob_13 + tprob_14 + tprob_15 + tprob_16 + tprob_17 + tprob_18 + tprob_19 + tprob_20 + tprob_21 + tprob_22 + tprob_23 + tprob_24 + tprob_25 + tprob_26 + tprob_27 + tprob_28 + tprob_29 + tprob_30 + tprob_31 + tprob_32 + tprob_33 + tprob_34 + tprob_35 + tprob_36 + tprob_37 + tprob_38 + tprob_39 + tprob_40 + tprob_41 + tprob_42 + tprob_43 + tprob_44'

In [51]:
panel = fore1Q.dropna(subset=tprob_cols + ['GDP', 'PCE', 'UN'])

In [52]:
import numpy as np

keep_cols = np.zeros(45)

for col in ['PCE', 'GDP', 'UN']:
    X = panel[tprob_cols]
    y = panel[col]

    counts = np.zeros(45)

    for i in range(100):
        from sklearn.model_selection import KFold
        kfold = KFold(n_splits=10, shuffle=True, random_state=i)
        reg = LassoCV(cv=kfold).fit(X,y)
        # print(reg.coef_)
        counts += (reg.coef_ != 0) * 1
    cols_for_topic = counts > 50
    print([i for i, val in enumerate(cols_for_topic) if val])

    keep_cols = np.logical_or(keep_cols, cols_for_topic)

print([i for i, val in enumerate(keep_cols) if val])
sum(keep_cols)


[0, 1, 2, 4, 7, 9, 10, 11, 13, 14, 16, 18, 22, 27, 28, 31, 32, 33, 36, 37, 40, 43]
[0, 1, 2, 4, 5, 6, 7, 8, 10, 14, 16, 18, 22, 27, 32, 33, 37, 38, 39, 41]
[0, 1, 2, 4, 5, 11, 12, 16, 18, 20, 22, 27, 29, 30, 31, 32, 34, 36, 37, 38, 40, 41, 43]
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 20, 22, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 43]


33

In [53]:
keep = [i for i, val in enumerate(keep_cols) if val == True]

In [54]:
keep

[0,
 1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 16,
 18,
 20,
 22,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 40,
 41,
 43]

In [55]:
def trim_svect(svect):
    new = np.zeros(len(keep))
    for i, index in enumerate(keep):
        new[i] = svect[index]
    return new

In [56]:
mfdff.columns

Index(['date_x', 'section', 'lname', 'svect', 'voter', 'sent', 'region',
       'female', 'chair', 'exp', 'exp_tertile', 'expd', 'date_section_mean',
       'date_section_std', 'norm_svect', 'temp_svect', 'temp_total',
       'total_exp', 'total_expd', 'num_expd', 'avg_expd', 'diff_exp_norm',
       'use', 'date', 'year', 'm', 'tprob_0', 'tprob_1', 'tprob_2', 'tprob_3',
       'tprob_4', 'tprob_5', 'tprob_6', 'tprob_7', 'tprob_8', 'tprob_9',
       'tprob_10', 'tprob_11', 'tprob_12', 'tprob_13', 'tprob_14', 'tprob_15',
       'tprob_16', 'tprob_17', 'tprob_18', 'tprob_19', 'tprob_20', 'tprob_21',
       'tprob_22', 'tprob_23', 'tprob_24', 'tprob_25', 'tprob_26', 'tprob_27',
       'tprob_28', 'tprob_29', 'tprob_30', 'tprob_31', 'tprob_32', 'tprob_33',
       'tprob_34', 'tprob_35', 'tprob_36', 'tprob_37', 'tprob_38', 'tprob_39',
       'tprob_40', 'tprob_41', 'tprob_42', 'tprob_43', 'tprob_44'],
      dtype='object')

In [57]:
mfdff['final_norm_svect'] = mfdff['norm_svect'].map(lambda x : trim_svect(x).tolist())
mfdff['final_diff_exp_norm'] = mfdff['diff_exp_norm'].map(lambda x : trim_svect(x).tolist())

In [59]:
mfdff['final_diff_exp_norm']

0       [3.01023467, 2.57114901, 2.11642485, 0.0, 2.90...
1       [2.13716221, 1.66424159, 2.11642485, -4.008918...
2       [1.35656806, 2.57114901, 1.7087177, 0.0, 2.900...
3       [-6.68206393, 7.56301231, 2.11642485, 0.0, 2.9...
4       [3.01023467, -2.85408946, 2.11642485, 0.0, 2.9...
                              ...                        
5880    [-2.84844268, 0.0, -1.85493179, -3.27615621, 0...
5881    [0.0, 0.0, -0.1854931, -0.2471833, 0.0, 0.5886...
5882    [0.0, 0.0, -0.1854931, -0.2471833, 0.0, 0.5886...
5883    [0.0, 0.0, -1.85493179, -2.47183336, 0.0, 5.88...
5884    [0.0, 0.0, -0.1854931, -0.2471833, 0.0, 0.5886...
Name: final_diff_exp_norm, Length: 5885, dtype: object

In [45]:
mfdff.to_csv('../working-csvs/mfdff_selected.csv')